In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv("../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")

In [ ]:
data

In [ ]:
print(data)

In [ ]:
#del data[]
#del data['creatinine_phosphokinase']
#del data['diabetes']
#del data['high_blood_pressure']
#del data['platelets']
#del data['smoking']
#del data['serum_creatinine']
#del data['serum_sodium']
#del data['sex']
#del data['time']


In [ ]:
fix_data = data.drop(['anaemia','creatinine_phosphokinase','diabetes','high_blood_pressure',
                     'platelets','smoking','serum_creatinine','serum_sodium','sex','time'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
data_latih,data_uji  = train_test_split(data,test_size=0.2,random_state=101)

In [ ]:
print(data_uji.shape[0])
print(data_latih.shape[0])

In [ ]:
data

In [ ]:
##kelas= fix_data.DEATH_EVENT
kelas= data_uji.DEATH_EVENT

# CLUSTERING

## Hierarichal

In [ ]:
uji_hierarchy = data_uji.iloc[0:10]

In [ ]:
X = uji_hierarchy[["age", "ejection_fraction"]].to_numpy() 

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt

linked = linkage(X, 'average')

labelList = range(1, 11)

plt.figure(figsize=(10, 7))
dendrogram(linked,
            orientation='top',
            labels=labelList,
            distance_sort='descending',
            show_leaf_counts=True)
plt.show()

In [ ]:
linked

## K-Means

In [ ]:
def euclidean(data1,data2):
  jarak = np.square(data1-data2)
  jarak = np.sum(jarak)
  return np.sqrt(jarak)


In [ ]:
jarak = euclidean(data_latih,data_uji)
print(jarak)

In [ ]:
from random import randint
import copy

def kmeans(data_input, k, iter_max=200):
  n_data = data_input.shape[0]
  n_fitur = data_input.shape[1]
  iter = 1
  cluster = np.array([randint(0, k-1) for i in range(n_data)])
  print(cluster)
  centroid = np.zeros((k, n_fitur))
  print(centroid)
  cluster_tidak_berubah = False
  
  while cluster_tidak_berubah == False and iter<iter_max:
    jarak = np.zeros((n_data, k))

    for i in range(k):
      centroid[i,:]=data_input.iloc[np.where(cluster==i)[0]].mean().tolist()

    for i in range(n_data):
      for j in range(k):
        jarak[i][j] = euclidean(data_input.iloc[i], centroid[j])
    
    cluster_baru = [np.argmin(jarak[i]) for i in range(n_data)]
    cluster_tidak_berubah = np.array_equal(cluster_baru, cluster)
    cluster = np.array(copy.deepcopy(cluster_baru))
    iter+=1

    print(jarak)
    print(cluster)
    print(centroid)
    print(cluster_baru)
    return(cluster_baru)

In [ ]:
cluster = kmeans(data_uji, 3)

In [ ]:
print(cluster)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(data_uji['age'], data_uji['ejection_fraction'], c=cluster)
plt.plot

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [ ]:
print(confusion_matrix(data_uji.DEATH_EVENT,cluster))

## KNN

In [ ]:
##from collections import Counter
##def knn(k,datalatih,labellatih,datauji):
##  jarak = np.array([euclidean(datalatih.iloc[x],datauji) for x in range(datalatih.shape[0])])
 ## indeks_k_minimum = jarak.argsort()[:k]
##  k_kelas = labellatih.iloc[indeks_k_minimum].to_numpy()
 ## counter = Counter(k_kelas)
 ## kelas_uji = counter.most_common(1)[0][0]
 ## return kelas_uji

In [ ]:
## hasil = knn(3,data_latih,label_latih,data_uji.iloc[1])
## print(hasil)

In [ ]:
##def knn_multi(k,datalatih,labellatih,datauji):
##  kelasuji = np.array([knn(k,datalatih,labellatih,datauji.iloc[x])for x in range(datauji.shape[0])])
 ## return kelasuji

In [ ]:
##def cek_hasil(prediksi, kebenaran):
 ## return sum(prediksi==kebenaran)

In [ ]:
##hasil = knn(3,data_latih,label_latih,data_uji.iloc[1])

In [ ]:
##prediksi1 = knn_multi(3, data_latih, label_latih, data_uji)

In [ ]:
##cek_hasil(prediksi1, label_uji)

## Evaluasi Clustering

In [ ]:
import timeit

from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [ ]:
startkmc = timeit.default_timer()
kmeans = KMeans(3,init='k-means++')
stopkmc = timeit.default_timer()
kmeans.fit(data_uji.drop('DEATH_EVENT',axis=1))
print(confusion_matrix(data_uji.DEATH_EVENT,kmeans.labels_))

In [ ]:
print(classification_report(data_uji.DEATH_EVENT,kmeans.labels_))


In [ ]:
f"Accuracy : {np.round(100*accuracy_score(data_uji.DEATH_EVENT,kmeans.labels_),2)}"

## EVALUASI KLASIFIKASI

In [ ]:
import timeit
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_uji.drop('DEATH_EVENT',axis=1),kelas)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
startknn = timeit.default_timer()
preds_knn = knn.predict(X_test)
stopknn = timeit.default_timer()
print(confusion_matrix(y_test,preds_knn))

In [ ]:
print(classification_report(y_test,preds_knn))


## Evaluasi KNN

In [ ]:
print(confusion_matrix(y_test,preds_knn))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


In [ ]:
plt.title("K Nearest Neighbors Confusion Matrix")
sns.heatmap(confusion_matrix(y_test,preds_knn),annot=True,cmap="Blues",fmt="d",cbar=False)

True Positive(TP) : 30

False Negative(FN) : 3

False Positive(FP) : 1

False Positive(TN) : 1